In [1]:
# Reading Data
import pandas as pd
import numpy as np
df_a = pd.read_csv('mean_psi.csv')
df_r = pd.read_csv("mean_lambda.csv")
df_a = np.exp(df_a.to_numpy())
df_r = np.exp(df_r.to_numpy())

In [2]:
# Parameters
M = 200
a0 = 0.15
b = np.array([1,2,9,1,2,9,1,2,9,1,2,9,1,2,9,1,2,9])
b = b.reshape((18,1))
# p = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20])
# p_new = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20]) ##自己選
e = np.random.normal(0,1,18) # will change
e = np.exp(e)    
# print(e)

In [3]:
from gurobipy import *
import gurobipy as gp

In [4]:
def new_situation(M, e, p):
    # M: budget
    # e: random utilty term for all the 18 rental options
    # p: price for all the 18 rental options

    # Construct Model
    m = gp.Model('m_(0,0)')
    x = m.addVars(18, name="x", vtype=GRB.INTEGER)
    y = m.addVars(18, name="y") #indide log term r*b*s
    lapprox = m.addVars(18, name="lapprox") #the whole log term
    m.addConstrs(x[j] * float(df_r[j] * b[j]) + 1 == y[j] for j in range(18))  #inside log r*b*s can sum up together
    for j in range(18):
        m.addGenConstrLog(y[j], lapprox[j])                                    #the whole log-- cannot sum up, need for loop
    m.addConstr(M - quicksum(p[j] * x[j] for j in range(18)) >= 0)
    m.setObjective(quicksum(float(df_a[j] * e[j]/df_r[j]) * lapprox[j]for j in range(18)) + a0 * M - a0 * quicksum(p[j] * x[j] for j in range(18)))
    
    # Solve model
    m.modelSense = GRB.MAXIMIZE
    m.setParam('OutputFlag', 0)
    m.params.threads = 1 
    m.optimize()
    
    # Retrive Quantity
    Q = []
    count = 0
    for j in m.getVars():   
        Q.append(j.x)
        count = count + 1
        if count == 18:
            break    
    q_array = np.array(Q)
    # Return Value
    return q_array

In [5]:
import os, sys
import statistics as stat

In [6]:
p_o = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20])
p_d = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20])
p_d = p_d.astype(float)
# discount = 0.2  #1 dollar discount

# only the first rental option is disounted, at 80 percent of the original price
discount = 4.9 
rentalOption = ( 1 ) - 1 

# the index of rental option
p_d[rentalOption] = p_d[rentalOption] - discount
# count_d = np.zeros(18)
# count_o = np.zeros(18)

n = 10
M = 200

Elas0 = list()
QoList = list()
QdList = list()

i = 0
count = 0
count0 = 0
while i < n:
    e = np.random.normal(0, 1, 18) # will change in each iteration
    e = np.exp(e)
    
#     count_d = count_d + new_situation(M, e, p_d)
#     count_o = count_o + new_situation(M, e, p_o)
    Qd = new_situation(M, e, p_d)[rentalOption]
    Qo = new_situation(M, e, p_o)[rentalOption]
    if Qo > 0 and Qd >= Qo + 0.001:
        Elas = ((Qd - Qo) / Qo) * (p_d[rentalOption] / discount)
        Elas0.append(Elas)
        QdList.append(Qd)
        QoList.append(Qo)
    else:
        continue
    i = i + 1

Elas0 = np.asarray(Elas0)
averageQo = stat.mean(QoList)
averageQd = stat.mean(QdList)
averageElasticity = ((averageQd - averageQo) / averageQo) * (p_d[rentalOption] / discount)

# print(Elas0)
print(averageElasticity)

Using license file /Users/luhsin/gurobi.lic
Academic license - for non-commercial use only - expires 2021-04-05
1
1.2681405895692162


In [33]:
def elasticity(p_o, p_d, discount, rentalOption, n, M):
    Elas0 = list()
    QoList = list()
    QdList = list()

    i = 0
    count = 0
    count0 = 0

    while i < n:
        e = np.random.normal(0, 1, 18) # will change
        e = np.exp(e)

        Qd = new_situation(M, e, p_d)[rentalOption]
        Qo = new_situation(M, e, p_o)[rentalOption]

        if Qo >= 0.001 and Qd >= Qo + 0.001:
            Elas = ((Qd - Qo) / Qo) * (p_d[rentalOption] / discount)
            Elas0.append(Elas)
            QdList.append(Qd)
            QoList.append(Qo)
            
        else:
            count = count + 1
            continue
        i = i + 1

    Elas0 = np.asarray(Elas0)
    averageQo = stat.mean(QoList)
    averageQd = stat.mean(QdList)
    averageElasticity = ((averageQd - averageQo) / averageQo) * (p_d[rentalOption] / discount)
    return [stat.mean(Elas0), stat.variance(Elas0)]

In [34]:
# new_situation(M, e, p_d)

# elasticity(p_o, p_d, discount, rentalOption, n, M)
elasticityMeanList = list()
elasticityVarianceList= list()

for rentalOptioni in range(18):
    p_o = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20])
    p_d = np.array([5,10,20,5,10,20,5,10,20,5,10,20,5,10,20,5,10,20])
    p_d = p_d.astype(float)
    discount = 4.9 
    rentalOption = rentalOptioni

    p_d[rentalOption] = p_d[rentalOption] - discount
    n = 200
    M = 200
    temp = elasticity(p_o, p_d, discount, rentalOption, n, M)
    elasticityMeanList.append(temp[0])
    elasticityVarianceList.append(temp[1])
    print("Rental option", rentalOptioni + 1, "mean: ", temp[0], "variance: ", temp[1])

Rental option 1 mean:  1.4107827927625967 variance:  0.10908944347818376
Rental option 2 mean:  1.2815980320701923 variance:  0.19759442009040237
Rental option 3 mean:  2.458115646258503 variance:  0.6963020666291335
Rental option 4 mean:  1.4712719029961951 variance:  0.1475589770012019
Rental option 5 mean:  1.2542642128282016 variance:  0.18372952003674903
Rental option 6 mean:  2.5713656462585 variance:  0.6208775215769454
Rental option 7 mean:  1.4649937548283454 variance:  0.15151762030683535
Rental option 8 mean:  1.2628075801751204 variance:  0.21888574399710664
Rental option 9 mean:  2.3674642857142856 variance:  0.7955233030317717
Rental option 10 mean:  1.449317067852754 variance:  0.14185209773623583
Rental option 11 mean:  1.2330582155940273 variance:  0.17811780952969072
Rental option 12 mean:  2.53412925170068 variance:  0.6835340737655606
Rental option 13 mean:  1.3212960400448641 variance:  0.08922357827598598
Rental option 14 mean:  1.2794750971821627 variance:  0.207

In [35]:
import pandas as pd
# data = [elasticityMeanList, elasticityVarianceList]
df_elasticity = pd.DataFrame({"Mean":elasticityMeanList, 
                 "Variance":elasticityVarianceList})
print(df.round(2))

    Mean  Variance
0   1.41      0.11
1   1.28      0.20
2   2.46      0.70
3   1.47      0.15
4   1.25      0.18
5   2.57      0.62
6   1.46      0.15
7   1.26      0.22
8   2.37      0.80
9   1.45      0.14
10  1.23      0.18
11  2.53      0.68
12  1.32      0.09
13  1.28      0.21
14  2.38      0.80
15  1.44      0.14
16  1.26      0.20
17  2.50      0.73
